In [1]:
import torch
import numpy as np

In [2]:
from datasets import SemEval
from torch import nn, optim
from bert import encode
from matplotlib import pyplot as plt
from copy import deepcopy
from sklearn.metrics import f1_score, classification_report
from math import ceil
from warnings import filterwarnings
from utils import plot_confusion_matrix, print_misclassifications

In [3]:
filterwarnings("ignore")

In [4]:
class Model(nn.Module):
    
    def __init__(self, input_size, intermediate_size, target_size):
        super().__init__()
        self.classifier = nn.Sequential(
            nn.Linear(input_size, intermediate_size),
            nn.ReLU(),
            nn.Linear(intermediate_size, target_size),
            nn.Softmax(dim=1)
        )
        self.cross_entropy = nn.CrossEntropyLoss()
        
    def forward(self, inputs, labels=None):
        probabilities = self.classifier(inputs)
        return probabilities, torch.argmax(probabilities, 1), self.cross_entropy(probabilities, labels) if labels is not None else None

In [5]:
def get_model(state_dict=None):
    torch.manual_seed(42)
    model = Model(768, 64, 3).cuda()
    if state_dict is not None:
        model.load_state_dict(state_dict)
    return model

In [6]:
def get_batch(step, batch_size, z):
    batches = ceil(len(z[0]) / batch_size)
    i = step % batches
    start, end = i * batch_size, min((i + 1) * batch_size, len(z[0]))
    return [a[start:end].cuda() for a in z]

In [7]:
training, testing = SemEval.get_subtask_a_datasets("data/semeval/")

In [8]:
training_inputs = encode(training.texts, training.targets, method="conditional-target", maximum_length=80).cuda(), torch.tensor(training.stances).cuda()
testing_inputs  = encode(testing.texts,  testing.targets,  method="conditional-target", maximum_length=80).cuda(), torch.tensor(testing.stances ).cuda()

In [9]:
model = get_model()
batch_size = 48
optimiser = optim.Adam(model.parameters(), lr=1e-4)
step, steps_since, early_stopping = 0, 0, 1000
evaluation_function = lambda true, predicted: f1_score(true, predicted, average="macro")

In [10]:
optimal_score, optimal_model_state_dict = 0., None

In [11]:
training_loss, testing_loss = [], []

In [ ]:
while steps_since < early_stopping:

    model.zero_grad()
    optimiser.zero_grad()
    model.train()

    inputs = get_batch(step, batch_size, training_inputs)
    _, _, loss = model(*inputs)
    training_loss.append(float(loss))

    loss.backward()
    optimiser.step()

    model.eval()

    _, predictions, loss = model(*testing_inputs)
    score = evaluation_function(testing.stances, predictions.cpu())
    testing_loss.append(float(loss))
    
    if score > optimal_score:
        optimal_score = score
        optimal_model_state_dict = deepcopy(model.state_dict())
        steps_since = 0
    else:
        steps_since += 1

    step += 1

In [ ]:
plt.style.use("seaborn")
plt.figure(figsize=(10, 7))
plt.plot(training_loss[::ceil(len(training) / batch_size)])
plt.plot(testing_loss[::ceil(len(training) / batch_size)])
plt.legend(["Training", "Testing"])
plt.xlabel("Epoch")
plt.ylabel("Cross-entropy Loss")
pass

In [ ]:
model = get_model(state_dict=optimal_model_state_dict)
model.eval()

In [ ]:
_, predictions, _ = model(*testing_inputs)

In [ ]:
print(classification_report(testing.stances, predictions.cpu()))

In [ ]:
plot_confusion_matrix(testing.stances, predictions.cpu())

In [ ]:
print_misclassifications(testing, predictions.cpu())